# CITY COVID INFO


Aim: Focus on One City, get covid-19 information. 

Please explore the information in https://coronavirus.data.gov.uk/details/cases

And check how their APIs work.

https://coronavirus.data.gov.uk/details/developers-guide/main-api



In [1]:
import requests
import urllib
import pandas as pd

# CITY LIST

In [2]:
city_list = ['Brighton and Hove',
 'Liverpool',
 'Sheffield',
 'London',
 'Norfolk',
 'Leeds',
 'Manchester',
 'East Hampshire',
 'Leicester',
 'Wolverhampton',
 'Newcastle upon Tyne',
 'Southampton',
 'Burnley',
 'Hertfordshire',
 'Birmingham',
 'Bournemouth, Christchurch and Poole']

In [3]:
def get_data(url):
    """Gets data from url and returns json response"""
    response = requests.get(endpoint, timeout=10)
    if response.status_code >= 400:
        raise RuntimeError(f'Request failed: { response.text }')
    return response.json()

# Prep request payload
endpoint = 'https://coronavirus.data.gov.uk/api/v1/data?'
structure = '{"date":"date"}'#,"newCases":"newCasesBySpecimenDate","cumNewCases":"cumCasesBySpecimenDate"}'
encoded_structure = urllib.parse.quote(structure)
areaType = 'ltla'

# Dict for each city result
results = {}

for city in (city for city in city_list if city != 'London'):
    params = f'filters=areaType={areaType};areaName={city}&structure={encoded_structure}'
    response = get_data(endpoint+params)
    results[city] = response['data']
    
areaType = 'region'
city = 'London'
params = f'filters=areaType={areaType};areaName={city}&structure={encoded_structure}'
response = get_data(endpoint+params)
results[city] = response['data']

result_df = pd.DataFrame()

rename_columns = {'city': 'CityName', 'date': 'Date', 'latestBy': 'DailyCases', 'confirmed': 'CummulativeCases', 'areaName': 'AreaName'}
csv_columns = ['Date','CityName','AreaName','DailyCases','CummulativeCases']

# Create dataframe from dict
for city in results.keys():
    results[city] = [{**x, 'city': city} for x in results[city]]
    city_df = pd.DataFrame(results[city])
    result_df = pd.concat([result_df, city_df], axis=0)
    
    # Create individual city csv
    city_df.rename(columns=rename_columns, inplace=True)
    city_df.to_csv(f'raw/{city}_DAILY_CASES.csv', columns = csv_columns)    
    
# Create all cities csv from dataframe
result_df.rename(columns=rename_columns, inplace=True)
result_df.to_csv('raw/ALL_CITY_DAILY_CASES.csv', columns = csv_columns)